In [1]:
!pip install -r requirements.txt

You should consider upgrading via the 'C:\Users\shars\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


<h2>Load csv</h2>
<p>carico il training set e il test set prodotti nel preparation notebook</p>

In [2]:
import pandas as pd

dataframe_path_train = "train.csv"
dataframe_path_test = "test.csv"

dataframe_train = pd.read_csv(dataframe_path_train)
dataframe_test = pd.read_csv(dataframe_path_test)

<h1>Studio dei dati</h1>
<p>Andiamo ad analizzare i dati per vedere le caratteristiche del dataset.</p>

<h2>Training set</h2>
<p>Incominciamo dando una veloce occhiata di insieme su come sono composti i dati di training</p>

In [3]:
dataframe_train

,DEATH_EVENT,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,1,65.0,1,160,1,20,0,327000.00,2.70,116,0,0,8
1,0,81.0,0,4540,0,35,0,231000.00,1.18,137,1,1,107
2,0,55.0,0,1820,0,38,0,270000.00,1.20,139,0,0,271
3,1,72.0,0,233,0,45,1,235000.00,2.50,135,0,0,115
4,0,65.0,1,305,0,25,0,298000.00,1.10,141,1,0,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0,67.0,0,213,0,38,0,215000.00,1.20,133,0,0,245
156,0,65.0,0,582,1,40,0,270000.00,1.00,138,0,0,140
157,0,50.0,1,582,1,20,1,279000.00,1.00,134,0,0,186
158,0,85.0,0,212,0,38,0,186000.00,0.90,136,1,0,187


<h2>Report con pandas profiling</h2>
<p><a href="https://github.com/pandas-profiling/pandas-profiling">Pandas Profiling</a> è un progetto molto utile che permette di generare un completo report che riassume molteplici aspetti di un dataframe pandas. Nel report vengono riportate le seguenti informazioni:</p>
<ol>
    <li>Tipo di dato di ogni feature.</li>
    <li>Presenza di valori nulli o mancanti.</li>
    <li>Statistiche per ogni feature, come valore medio, massimo, minimo...</li>
    <li>Correlazioni tra le variabili</li>
</ol>
<p><i>Ho lasciato commentato la riga con la visualizzazione del report per evitare che potesse consumare  tempo durante l'esecuzione del notebook</i></p>

In [4]:
from pandas_profiling import ProfileReport

report = ProfileReport(dataframe_train, title="Report")
#report

<h3>Considerazioni</h3>
<p>Possiamo notare come la totalità dei dati è booleano o numerica</p>
<p>Salta all'occhio l'importanza del campo "time" che risulta essere abbastanza indicativo della possibilità che un paziente sia deceduto o no.</p>

<h2>Controllo manuale</h2>
<p>Controllo manualmente se è evidente in che modo le varie feature influiscono in "DEATH_EVENT".</p>

In [5]:
pd.set_option('max_columns', None)

for col in dataframe_train:
    print(f"\n\n{col}")
    print(pd.crosstab(dataframe_train["DEATH_EVENT"], dataframe_train[col], rownames=['death'],colnames=['valore']))



DEATH_EVENT
valore    0   1
death          
0       113   0
1         0  47


age
valore  40.000  41.000  42.000  43.000  44.000  45.000  46.000  47.000  \
death                                                                    
0            1       1       2       1       1       8       2       1   
1            0       0       1       0       0       0       1       0   

valore  50.000  51.000  52.000  53.000  55.000  57.000  58.000  59.000  \
death                                                                    
0            9       3       5       5       6       1       5       1   
1            5       0       0       1       1       0       1       2   

valore  60.000  60.667  61.000  62.000  63.000  64.000  65.000  67.000  \
death                                                                    
0           11       1       2       3       5       1      10       1   
1            6       1       0       1       0       0       7       0   

valore  68.000  69.000  7

valore  116  124  125  126  127  128  129  130  131  132  133  134  135  136  \
death                                                                          
0         0    0    1    1    0    1    0    3    2    3    7    9    7   15   
1         1    1    0    0    2    1    2    0    1    4    1    9    3    7   

valore  137  138  139  140  141  142  143  144  145  146  
death                                                     
0        12    9    8   15   10    2    1    3    4    0  
1         3    4    1    4    0    0    0    1    1    1  


sex
valore   0   1
death         
0       41  72
1       18  29


smoking
valore   0   1
death         
0       77  36
1       34  13


time
valore  6    7    8    10   11   13   14   16   20   22   24   26   27   28   \
death                                                                          
0         0    0    0    0    0    0    0    1    0    1    0    0    0    0   
1         1    2    1    3    1    1    1    0    1    0    

<h1>Modello</h1>
<p>Ho sfruttato sklearn per creare e utilizzare il modello</p>
<p>La task assegnata è risolvibile con un albero decisionale</p>
<h2>Random Forest</h2>
<p>Si è scelto l'algoritmo Random Forest poiché i risultati di questo si rivelano migliori dei normali alberi decisionali.</p>
<h3>Descrizione di Random Forest</h3>
<p>Invece di costruire un singolo albero di decisione prevede la creazione di una multitudine di alberi decisionali. Successivamente, quando si dovrà effettuare una previsione, si useranno gli input su più alberi, l'algoritmo decide democraticamente il risultato scegliendo il voto di maggioranza.</p>
<p>Per creare più alberi si sfrutta il fatto che gli alberi decisionali sono sensibili ai dati di training (cambiando anche di poco i dati di training l'albero potrebbe avere una struttura molto diversa). Si effettuerà un operazione chiamata Bagging (bootstrapping + aggregate) che consiste nel prelevare casulmente dai dati di training degli esempi (anche più volte lo stesso) e utilizzare questi  per il calcolo della split feature (sklearn usa di default la Giny inpurity, ma è possibile anche usare l'entropia). Questa operazione viene ripetuta per la crazione di vari alberi diversi. Inoltre in ogni albero potrebbero essere considerate non tutte le feature (magari solo un paio), in modo che i vari alberi possono prendere decisioni basandosi su feature diverse.</p>
<p>In questo modo si riescono a superare alcuni errori dovuti ad errori causati magari da alberi isolati.</p>

<h2>Esecuzione</h2>
<p>Andiamo effettivamente ad implementare, addestrare e testare il modello.</p>

<h3>Creazione del vettore target</h3>

Preleviamo i valori della "DEATH_EVENT" e li mettiamo all'interno di un vettore per poi utilizzarli in fase di training e di test.

In [6]:
target_feature = "DEATH_EVENT"
dataframe_train_target = dataframe_train.pop(target_feature)
dataframe_test_target = dataframe_test.pop(target_feature)

<h3>Training</h3>
<p>Creo e addestro il modello.</p>

In [7]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()\
                .fit(dataframe_train, dataframe_train_target)

<h3>Testing</h3>
<p>Andiamo ad utilizzare i dati di test</p>

In [8]:
predictions = classifier.predict(dataframe_test)

<h3>Risultati</h3>
<p>Andiamo a stampare precision, recall, accuracy e la matrice di confusione.</p>

In [9]:
from sklearn import metrics

accuracy = metrics.accuracy_score(dataframe_test_target, predictions)
precision = metrics.precision_score(dataframe_test_target, predictions)
recall = metrics.recall_score(dataframe_test_target, predictions)

print(f"precision: {precision:.4} -- recall: {recall:.4} -- accuracy: {accuracy:.4} ")
print(pd.crosstab(dataframe_test_target, predictions, rownames=['valore effettivo'], colnames=['valore previsto']))

precision: 0.9474 -- recall: 0.75 -- accuracy: 0.9114 
valore previsto    0   1
valore effettivo        
0                 54   1
1                  6  18
